<a href="https://colab.research.google.com/github/IguruPeter/Medical-Symptom-Checker/blob/main/SymptomScreeningChatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
from typing import Dict, List

class SymptomChatbot:
    def __init__(self):
        # Enhanced symptom database with detailed medical information
        self.symptom_db = {
            "fever": {
                "severity": "high",
                "questions": [
                    "How long have you had fever?",
                    "What is your temperature?",
                    "Is it constant or intermittent?",
                    "Have you experienced any chills?"
                ],
                "red_flags": ["temperature > 103°F", "fever with stiff neck", "fever with confusion"],
                "related_symptoms": ["chills", "sweating", "headache"]
            },
            "headache": {
                "severity": "medium",
                "questions": [
                    "How severe is the headache?",
                    "When did it start?",
                    "Is it constant or does it come and go?",
                    "Is it affecting your vision?",
                    "Is it worse with movement?"
                ],
                "red_flags": ["sudden severe headache", "headache with confusion", "headache with fever"],
                "related_symptoms": ["nausea", "vomiting", "sensitivity to light"]
            },
            "cough": {
                "severity": "medium",
                "questions": [
                    "Is it dry or productive?",
                    "How long have you had it?",
                    "What color is any mucus?",
                    "Is there any blood?",
                    "Do you have chest pain when coughing?"
                ],
                "red_flags": ["blood in cough", "severe chest pain", "difficulty breathing"],
                "related_symptoms": ["fever", "fatigue", "chest tightness"]
            },
            "chest_pain": {
                "severity": "high",
                "questions": [
                    "Where exactly is the pain?",
                    "How long have you had it?",
                    "Does it move to other areas?",
                    "Is it constant or does it come and go?",
                    "Does it change with breathing or movement?"
                ],
                "red_flags": ["severe chest pain", "pain radiating to arm/jaw", "difficulty breathing"],
                "related_symptoms": ["shortness of breath", "nausea", "dizziness"]
            },
            "abdominal_pain": {
                "severity": "medium",
                "questions": [
                    "Where is the pain located?",
                    "How long have you had it?",
                    "Is it constant or does it come and go?",
                    "Does it radiate to other areas?",
                    "Is it related to eating?"
                ],
                "red_flags": ["severe pain", "vomiting blood", "inability to pass gas"],
                "related_symptoms": ["nausea", "vomiting", "diarrhea"]
            }
        }

        # Enhanced triage logic thresholds
        self.triage_thresholds = {
            "emergency": {
                "score": 8,
                "red_flags": ["severe chest pain", "difficulty breathing", "confusion"],
                "message": "Seek immediate emergency care"
            },
            "urgent": {
                "score": 5,
                "message": "See a doctor within 24 hours"
            },
            "routine": {
                "score": 3,
                "message": "Schedule a routine appointment"
            }
        }

    def calculate_severity_score(self, symptoms: List[str], red_flags: List[str]) -> int:
        """Calculate severity score based on symptoms and red flags"""
        score = 0

        # Base symptom scores
        for symptom in symptoms:
            if symptom in self.symptom_db:
                severity = self.symptom_db[symptom]["severity"]
                if severity == "high":
                    score += 3
                elif severity == "medium":
                    score += 2
                else:
                    score += 1

        # Red flag multiplier
        for flag in red_flags:
            if any(flag in self.symptom_db[symptom]["red_flags"] for symptom in symptoms):
                score *= 2

        # Related symptom bonus
        for symptom in symptoms:
            if symptom in self.symptom_db:
                related = self.symptom_db[symptom]["related_symptoms"]
                score += sum(1 for s in symptoms if s in related)

        return score

    def process_symptoms(self, symptoms: List[str]) -> Dict:
        """Process symptoms and return triage recommendation"""
        red_flags = [flag for symptom in symptoms
                     for flag in self.symptom_db[symptom]["red_flags"]
                     if flag in symptoms]

        severity_score = self.calculate_severity_score(symptoms, red_flags)

        # Determine triage level
        if severity_score >= self.triage_thresholds["emergency"]["score"]:
            return {"triage_level": "emergency", "message": self.triage_thresholds["emergency"]["message"]}
        elif severity_score >= self.triage_thresholds["urgent"]["score"]:
            return {"triage_level": "urgent", "message": self.triage_thresholds["urgent"]["message"]}
        else:
            return {"triage_level": "routine", "message": self.triage_thresholds["routine"]["message"]}

    def get_followup_questions(self, symptoms: List[str]) -> List[str]:
        """Get relevant follow-up questions"""
        questions = []
        for symptom in symptoms:
            if symptom in self.symptom_db:
                questions.extend(self.symptom_db[symptom]["questions"])
        return questions

# Example usage
chatbot = SymptomChatbot()

# Test with different symptom combinations
test_cases = [
    ["fever", "headache"],  # High severity combination
    ["cough"],  # Single medium severity symptom
    ["chest_pain"],  # High severity symptom
    ["abdominal_pain"],  # Medium severity symptom
    ["fever", "cough", "headache"]  # Multiple symptoms
]

for symptoms in test_cases:
    print(f"\nSymptoms reported: {', '.join(symptoms)}")
    result = chatbot.process_symptoms(symptoms)
    print(f"Triage level: {result['triage_level']}")
    print(f"Message: {result['message']}")
    print("Follow-up questions:")
    for question in chatbot.get_followup_questions(symptoms):
        print(f"- {question}")

print("\nTesting red flags:")
red_flag_test = ["difficulty breathing"]
result = chatbot.process_symptoms(["chest_pain"])
print(f"Triage level: {result['triage_level']}")
print(f"Message: {result['message']}")

print("\nTesting multiple symptoms:")
multi_symptom_test = ["fever", "headache", "cough"]
result = chatbot.process_symptoms(multi_symptom_test)
print(f"Triage level: {result['triage_level']}")
print(f"Message: {result['message']}")


Symptoms reported: fever, headache
Triage level: urgent
Message: See a doctor within 24 hours
Follow-up questions:
- How long have you had fever?
- What is your temperature?
- Is it constant or intermittent?
- Have you experienced any chills?
- How severe is the headache?
- When did it start?
- Is it constant or does it come and go?
- Is it affecting your vision?
- Is it worse with movement?

Symptoms reported: cough
Triage level: routine
Message: Schedule a routine appointment
Follow-up questions:
- Is it dry or productive?
- How long have you had it?
- What color is any mucus?
- Is there any blood?
- Do you have chest pain when coughing?

Symptoms reported: chest_pain
Triage level: routine
Message: Schedule a routine appointment
Follow-up questions:
- Where exactly is the pain?
- How long have you had it?
- Does it move to other areas?
- Is it constant or does it come and go?
- Does it change with breathing or movement?

Symptoms reported: abdominal_pain
Triage level: routine
Messag

In [ ]:
# Create a function to handle user input
def get_user_input():
    """Get user input for symptoms and red flags"""
    print("\n=== Symptom Screening Chatbot ===")
    print("Please enter your symptoms (comma-separated):")
    symptoms = input("> ").lower().split(",")
    symptoms = [s.strip() for s in symptoms]

    print("\nAny red flags? (comma-separated, or press Enter to skip):")
    red_flags = input("> ").lower().split(",")
    red_flags = [f.strip() for f in red_flags]

    return symptoms, red_flags

# Example interaction
chatbot = SymptomChatbot()
symptoms, red_flags = get_user_input()

# Process and display results
result = chatbot.process_symptoms(symptoms)
print("\n=== Triage Results ===")
print(f"Triage Level: {result['triage_level']}")
print(f"Recommendation: {result['message']}")

print("\n=== Follow-up Questions ===")
for question in chatbot.get_followup_questions(symptoms):
    print(f"- {question}")

# Example usage:
# Enter symptoms: fever, headache
# Enter red flags: (press Enter to skip)

NameError: name 'SymptomChatbot' is not defined

In [ ]:
class SymptomChatbot:
    def __init__(self):
        # Enhanced symptom database with Swahili translations
        self.symptom_db = {
            "fever": {
                "severity": "high",
                "questions": [
                    "How long have you had fever?",
                    "What is your temperature?",
                    "Is it constant or intermittent?",
                    "Have you experienced any chills?"
                ],
                "swahili": {
                    "name": "homa",
                    "questions": [
                        "Muda gani umekuwa na homa?",
                        "Ni kiwango gani cha joto lako?",
                        "Ni mara kwa mara au inabadilika?",
                        "Umecheka kichwa?"
                    ],
                    "red_flags": [
                        "joto linalozidi digrii 40",
                        "homa na kichwa kinachouma",
                        "homa na kuchanganyikiwa"
                    ]
                },
                "red_flags": [
                    "temperature > 103°F",
                    "fever with stiff neck",
                    "fever with confusion"
                ]
            },
            "headache": {
                "severity": "medium",
                "questions": [
                    "How severe is the headache?",
                    "When did it start?",
                    "Is it constant or does it come and go?",
                    "Is it affecting your vision?",
                    "Is it worse with movement?"
                ],
                "swahili": {
                    "name": "kifua",
                    "questions": [
                        "Ni kiwango gani cha kifua?",
                        "Limeanza lini?",
                        "Ni mara kwa mara au inabadilika?",
                        "Inakusumbua macho?",
                        "Inakua zaidi unapoenda?"
                    ],
                    "red_flags": [
                        "kifua kali sana",
                        "kifua na kuchanganyikiwa",
                        "kifua na homa"
                    ]
                },
                "red_flags": [
                    "sudden severe headache",
                    "headache with confusion",
                    "headache with fever"
                ]
            }
        }

        # Enhanced triage logic thresholds
        self.triage_thresholds = {
            "emergency": {
                "score": 8,
                "red_flags": ["severe chest pain", "difficulty breathing", "confusion"],
                "message": "Seek immediate emergency care"
            },
            "urgent": {
                "score": 5,
                "message": "See a doctor within 24 hours"
            },
            "routine": {
                "score": 3,
                "message": "Schedule a routine appointment"
            }
        }

    def select_language(self):
        """Allow user to select language"""
        print("\nChagua lugha / Select language:")
        print("1. English")
        print("2. Swahili")
        choice = input("Chagua chaguo lako / Enter your choice (1/2): ")

        if choice == "2":
            self.current_language = "swahili"
        else:
            self.current_language = "english"

    def get_question(self, symptom, question):
        """Get question in current language"""
        if self.current_language == "swahili":
            return self.symptom_db[symptom]["swahili"]["questions"][question]
        return self.symptom_db[symptom]["questions"][question]

    def get_symptom_name(self, symptom):
        """Get symptom name in current language"""
        if self.current_language == "swahili":
            return self.symptom_db[symptom]["swahili"]["name"]
        return symptom

    def calculate_severity_score(self, symptoms: List[str], red_flags: List[str]) -> int:
        """Calculate severity score based on symptoms and red flags"""
        score = 0

        # Base symptom scores
        for symptom in symptoms:
            if symptom in self.symptom_db:
                severity = self.symptom_db[symptom]["severity"]
                if severity == "high":
                    score += 3
                elif severity == "medium":
                    score += 2
                else:
                    score += 1

        # Red flag multiplier
        for flag in red_flags:
            if any(flag in self.symptom_db[symptom].get("red_flags", []) for symptom in symptoms):
                score *= 2

        # Related symptom bonus
        for symptom in symptoms:
            if symptom in self.symptom_db:
                related = self.symptom_db[symptom].get("related_symptoms", [])
                score += sum(1 for s in symptoms if s in related)

        return score

    def process_symptoms(self, symptoms: List[str]) -> Dict:
        """Process symptoms and return triage recommendation"""
        red_flags = []
        for symptom in symptoms:
            if symptom in self.symptom_db:
                symptom_red_flags = self.symptom_db[symptom].get("red_flags", [])
                red_flags.extend([flag for flag in symptom_red_flags if flag in symptoms])

        severity_score = self.calculate_severity_score(symptoms, red_flags)

        # Determine triage level
        if severity_score >= self.triage_thresholds["emergency"]["score"]:
            return {"triage_level": "emergency", "message": self.triage_thresholds["emergency"]["message"]}
        elif severity_score >= self.triage_thresholds["urgent"]["score"]:
            return {"triage_level": "urgent", "message": self.triage_thresholds["urgent"]["message"]}
        else:
            return {"triage_level": "routine", "message": self.triage_thresholds["routine"]["message"]}

    def get_followup_questions(self, symptoms: List[str]) -> List[str]:
        """Get relevant follow-up questions"""
        questions = []
        for symptom in symptoms:
            if symptom in self.symptom_db:
                questions.extend(self.symptom_db[symptom]["questions"])
        return questions

# Example usage
chatbot = SymptomChatbot()
chatbot.select_language()

# Test with symptoms
symptoms = ["fever", "headache"]
result = chatbot.process_symptoms(symptoms)

print("\n=== Matokeo / Results ===")
print(f"Lugha / Language: {chatbot.current_language}")
print(f"Uchunguzi / Triage Level: {result['triage_level']}")
print(f"Ushauri / Recommendation: {result['message']}")

print("\n=== Maswali ya Ziada / Follow-up Questions ===")
for question in chatbot.get_followup_questions(symptoms):
    print(f"- {question}")


Chagua lugha / Select language:
1. English
2. Swahili
Chagua chaguo lako / Enter your choice (1/2): 1

=== Matokeo / Results ===
Lugha / Language: english
Uchunguzi / Triage Level: urgent
Ushauri / Recommendation: See a doctor within 24 hours

=== Maswali ya Ziada / Follow-up Questions ===
- How long have you had fever?
- What is your temperature?
- Is it constant or intermittent?
- Have you experienced any chills?
- How severe is the headache?
- When did it start?
- Is it constant or does it come and go?
- Is it affecting your vision?
- Is it worse with movement?


In [ ]:
class SymptomChatbot:
    def __init__(self):
        # Enhanced symptom database with Swahili translations
        self.symptom_db = {
            "fever": {
                "severity": "high",
                "questions": [
                    "How long have you had fever?",
                    "What is your temperature?",
                    "Is it constant or intermittent?",
                    "Have you experienced any chills?"
                ],
                "swahili": {
                    "name": "homa",
                    "questions": [
                        "Muda gani umekuwa na homa?",
                        "Ni kiwango gani cha joto lako?",
                        "Ni mara kwa mara au inabadilika?",
                        "Umecheka kichwa?"
                    ],
                    "red_flags": [
                        "joto linalozidi digrii 40",
                        "homa na kichwa kinachouma",
                        "homa na kuchanganyikiwa"
                    ]
                },
                "red_flags": [
                    "temperature > 103°F",
                    "fever with stiff neck",
                    "fever with confusion"
                ]
            },
            "headache": {
                "severity": "medium",
                "questions": [
                    "How severe is the headache?",
                    "When did it start?",
                    "Is it constant or does it come and go?",
                    "Is it affecting your vision?",
                    "Is it worse with movement?"
                ],
                "swahili": {
                    "name": "kifua",
                    "questions": [
                        "Ni kiwango gani cha kifua?",
                        "Limeanza lini?",
                        "Ni mara kwa mara au inabadilika?",
                        "Inakusumbua macho?",
                        "Inakua zaidi unapoenda?"
                    ],
                    "red_flags": [
                        "kifua kali sana",
                        "kifua na kuchanganyikiwa",
                        "kifua na homa"
                    ]
                },
                "red_flags": [
                    "sudden severe headache",
                    "headache with confusion",
                    "headache with fever"
                ]
            }
        }

        # Enhanced triage logic thresholds
        self.triage_thresholds = {
            "emergency": {
                "score": 8,
                "red_flags": ["severe chest pain", "difficulty breathing", "confusion"],
                "message": "Seek immediate emergency care"
            },
            "urgent": {
                "score": 5,
                "message": "See a doctor within 24 hours"
            },
            "routine": {
                "score": 3,
                "message": "Schedule a routine appointment"
            }
        }

    def select_language(self):
        """Allow user to select language"""
        print("\nChagua lugha / Select language:")
        print("1. English")
        print("2. Swahili")
        choice = input("Chagua chaguo lako / Enter your choice (1/2): ")

        if choice == "2":
            self.current_language = "swahili"
        else:
            self.current_language = "english"

    def get_question(self, symptom, question):
        """Get question in current language"""
        if self.current_language == "swahili":
            return self.symptom_db[symptom]["swahili"]["questions"][question]
        return self.symptom_db[symptom]["questions"][question]

    def get_symptom_name(self, symptom):
        """Get symptom name in current language"""
        if self.current_language == "swahili":
            return self.symptom_db[symptom]["swahili"]["name"]
        return symptom

    def calculate_severity_score(self, symptoms: List[str], red_flags: List[str]) -> int:
        """Calculate severity score based on symptoms and red flags"""
        score = 0

        # Base symptom scores
        for symptom in symptoms:
            if symptom in self.symptom_db:
                severity = self.symptom_db[symptom]["severity"]
                if severity == "high":
                    score += 3
                elif severity == "medium":
                    score += 2
                else:
                    score += 1

        # Red flag multiplier
        for flag in red_flags:
            if any(flag in self.symptom_db[symptom].get("red_flags", []) for symptom in symptoms):
                score *= 2

        # Related symptom bonus
        for symptom in symptoms:
            if symptom in self.symptom_db:
                related = self.symptom_db[symptom].get("related_symptoms", [])
                score += sum(1 for s in symptoms if s in related)

        return score

    def process_symptoms(self, symptoms: List[str]) -> Dict:
        """Process symptoms and return triage recommendation"""
        red_flags = []
        for symptom in symptoms:
            if symptom in self.symptom_db:
                symptom_red_flags = self.symptom_db[symptom].get("red_flags", [])
                red_flags.extend([flag for flag in symptom_red_flags if flag in symptoms])

        severity_score = self.calculate_severity_score(symptoms, red_flags)

        # Determine triage level
        if severity_score >= self.triage_thresholds["emergency"]["score"]:
            return {"triage_level": "emergency", "message": self.triage_thresholds["emergency"]["message"]}
        elif severity_score >= self.triage_thresholds["urgent"]["score"]:
            return {"triage_level": "urgent", "message": self.triage_thresholds["urgent"]["message"]}
        else:
            return {"triage_level": "routine", "message": self.triage_thresholds["routine"]["message"]}

    def get_followup_questions(self, symptoms: List[str]) -> List[str]:
        """Get relevant follow-up questions"""
        questions = []
        for symptom in symptoms:
            if symptom in self.symptom_db:
                questions.extend(self.symptom_db[symptom]["questions"])
        return questions

    def handle_followup(self, symptoms: List[str], user_response: str) -> Dict:
        """Handle follow-up questions and responses"""
        result = self.process_symptoms(symptoms)

        print("\n=== Matokeo / Results ===")
        print(f"Lugha / Language: {self.current_language}")
        print(f"Uchunguzi / Triage Level: {result['triage_level']}")
        print(f"Ushauri / Recommendation: {result['message']}")

        print("\n=== Maswali ya Ziada / Follow-up Questions ===")
        for question in self.get_followup_questions(symptoms):
            print(f"- {question}")

        return result

# Example usage
chatbot = SymptomChatbot()
chatbot.select_language()

# Test with symptoms
symptoms = ["fever", "headache"]
result = chatbot.handle_followup(symptoms, "I have both symptoms")

print("\n=== Jibu la Mtumiaji / User Response ===")
print(f"Jibu / Response: {result['message']}")


Chagua lugha / Select language:
1. English
2. Swahili
Chagua chaguo lako / Enter your choice (1/2): 1

=== Matokeo / Results ===
Lugha / Language: english
Uchunguzi / Triage Level: urgent
Ushauri / Recommendation: See a doctor within 24 hours

=== Maswali ya Ziada / Follow-up Questions ===
- How long have you had fever?
- What is your temperature?
- Is it constant or intermittent?
- Have you experienced any chills?
- How severe is the headache?
- When did it start?
- Is it constant or does it come and go?
- Is it affecting your vision?
- Is it worse with movement?

=== Jibu la Mtumiaji / User Response ===
Jibu / Response: See a doctor within 24 hours


In [ ]:
chatbot = SymptomChatbot()

In [ ]:
chatbot.select_language()


Chagua lugha / Select language:
1. English
2. Swahili
Chagua chaguo lako / Enter your choice (1/2): 2


In [ ]:
symptoms = ["fever", "headache"]
result = chatbot.handle_followup(symptoms, "I have both symptoms")


=== Matokeo / Results ===
Lugha / Language: swahili
Uchunguzi / Triage Level: urgent
Ushauri / Recommendation: See a doctor within 24 hours

=== Maswali ya Ziada / Follow-up Questions ===
- How long have you had fever?
- What is your temperature?
- Is it constant or intermittent?
- Have you experienced any chills?
- How severe is the headache?
- When did it start?
- Is it constant or does it come and go?
- Is it affecting your vision?
- Is it worse with movement?


In [ ]:
# Create a function to handle user input
def get_user_input():
    """Get user input for symptoms and red flags"""
    print("\n=== Symptom Screening Chatbot ===")
    print("Please enter your symptoms (comma-separated):")
    symptoms = input("> ").lower().split(",")
    symptoms = [s.strip() for s in symptoms]

    print("\nAny red flags? (comma-separated, or press Enter to skip):")
    red_flags = input("> ").lower().split(",")
    red_flags = [f.strip() for f in red_flags]

    return symptoms, red_flags

# Example interaction
chatbot = SymptomChatbot()
symptoms, red_flags = get_user_input()

# Process and display results
result = chatbot.process_symptoms(symptoms)
print("\n=== Triage Results ===")
print(f"Triage Level: {result['triage_level']}")
print(f"Recommendation: {result['message']}")

print("\n=== Follow-up Questions ===")
for question in chatbot.get_followup_questions(symptoms):
    print(f"- {question}")

# Example usage:
# Enter symptoms: fever, headache
# Enter red flags: (press Enter to skip)


=== Symptom Screening Chatbot ===
Please enter your symptoms (comma-separated):
> headache, fever, cough

Any red flags? (comma-separated, or press Enter to skip):
> high fever

=== Triage Results ===
Triage Level: urgent
Recommendation: See a doctor within 24 hours

=== Follow-up Questions ===
- How severe is the headache?
- When did it start?
- Is it constant or does it come and go?
- Is it affecting your vision?
- Is it worse with movement?
- How long have you had fever?
- What is your temperature?
- Is it constant or intermittent?
- Have you experienced any chills?
